In [125]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

import matplotlib.pyplot as plt
import pandas as pd

In [126]:
df=pd.read_csv("matches.csv",index_col=0)

In [127]:
df.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0.0,1.0,Tottenham,...,Match Report,NaN,18.0,4.0,16.9,1.0,0.0,0.0,2022,Manchester City
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5.0,0.0,Norwich City,...,Match Report,NaN,16.0,4.0,17.3,1.0,0.0,0.0,2022,Manchester City
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5.0,0.0,Arsenal,...,Match Report,NaN,25.0,10.0,14.3,0.0,0.0,0.0,2022,Manchester City
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1.0,0.0,Leicester City,...,Match Report,NaN,25.0,8.0,14.0,0.0,0.0,0.0,2022,Manchester City
6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0.0,0.0,Southampton,...,Match Report,NaN,16.0,1.0,15.7,1.0,0.0,0.0,2022,Manchester City


In [128]:
# 38 matches x 20 teams x 2 seasons
#to investigate the missing data
38*20*2

1520

In [129]:
#see how many matches each team played
df["team"].value_counts()

,count
team,
Manchester United,72
West Ham United,72
Newcastle United,72
Brighton and Hove Albion,72
Southampton,72
Tottenham Hotspur,71
Manchester City,71
Leeds United,71
Wolverhampton Wanderers,71


In [130]:
df["round"].value_counts()

,count
round,
Matchweek 1,39
Matchweek 2,39
Matchweek 3,39
Matchweek 4,39
Matchweek 5,39
Matchweek 6,39
Matchweek 7,39
Matchweek 8,39
Matchweek 9,39


In [131]:
# cleaning our data
#check the types of data
df.dtypes

,0
date,object
time,object
comp,object
round,object
day,object
venue,object
result,object
gf,float64
ga,float64
opponent,object


In [132]:
df["date"]=pd.to_datetime(df["date"]) #convert the date data type

In [133]:
df.dtypes

,0
date,datetime64[ns]
time,object
comp,object
round,object
day,object
venue,object
result,object
gf,float64
ga,float64
opponent,object


In [134]:
#convert venue to a numericvalue
df["venue_code"]=df["venue"].astype("category").cat.codes
#venue code will indicates if the team was home or away by 0 and 1
df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,notes,sh,sot,dist,fk,pk,pkatt,season,team,venue_code
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0.0,1.0,Tottenham,...,NaN,18.0,4.0,16.9,1.0,0.0,0.0,2022,Manchester City,0
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5.0,0.0,Norwich City,...,NaN,16.0,4.0,17.3,1.0,0.0,0.0,2022,Manchester City,1
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5.0,0.0,Arsenal,...,NaN,25.0,10.0,14.3,0.0,0.0,0.0,2022,Manchester City,1
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1.0,0.0,Leicester City,...,NaN,25.0,8.0,14.0,0.0,0.0,0.0,2022,Manchester City,0
6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0.0,0.0,Southampton,...,NaN,16.0,1.0,15.7,1.0,0.0,0.0,2022,Manchester City,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2021-05-02,19:15,Premier League,Matchweek 34,Sun,Away,L,0.0,4.0,Tottenham,...,NaN,8.0,1.0,17.4,0.0,0.0,0.0,2021,Sheffield United,0
39,2021-05-08,15:00,Premier League,Matchweek 35,Sat,Home,L,0.0,2.0,Crystal Palace,...,NaN,7.0,0.0,11.4,1.0,0.0,0.0,2021,Sheffield United,1
40,2021-05-16,19:00,Premier League,Matchweek 36,Sun,Away,W,1.0,0.0,Everton,...,NaN,10.0,3.0,17.0,0.0,0.0,0.0,2021,Sheffield United,0
41,2021-05-19,18:00,Premier League,Matchweek 37,Wed,Away,L,0.0,1.0,Newcastle Utd,...,NaN,11.0,1.0,16.0,1.0,0.0,0.0,2021,Sheffield United,0


In [135]:
df["opponent_code"]=df["opponent"].astype("category").cat.codes
df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,sh,sot,dist,fk,pk,pkatt,season,team,venue_code,opponent_code
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0.0,1.0,Tottenham,...,18.0,4.0,16.9,1.0,0.0,0.0,2022,Manchester City,0,18
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5.0,0.0,Norwich City,...,16.0,4.0,17.3,1.0,0.0,0.0,2022,Manchester City,1,15
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5.0,0.0,Arsenal,...,25.0,10.0,14.3,0.0,0.0,0.0,2022,Manchester City,1,0
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1.0,0.0,Leicester City,...,25.0,8.0,14.0,0.0,0.0,0.0,2022,Manchester City,0,10
6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0.0,0.0,Southampton,...,16.0,1.0,15.7,1.0,0.0,0.0,2022,Manchester City,1,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2021-05-02,19:15,Premier League,Matchweek 34,Sun,Away,L,0.0,4.0,Tottenham,...,8.0,1.0,17.4,0.0,0.0,0.0,2021,Sheffield United,0,18
39,2021-05-08,15:00,Premier League,Matchweek 35,Sat,Home,L,0.0,2.0,Crystal Palace,...,7.0,0.0,11.4,1.0,0.0,0.0,2021,Sheffield United,1,6
40,2021-05-16,19:00,Premier League,Matchweek 36,Sun,Away,W,1.0,0.0,Everton,...,10.0,3.0,17.0,0.0,0.0,0.0,2021,Sheffield United,0,7
41,2021-05-19,18:00,Premier League,Matchweek 37,Wed,Away,L,0.0,1.0,Newcastle Utd,...,11.0,1.0,16.0,1.0,0.0,0.0,2021,Sheffield United,0,14


In [136]:
#removing : and minute from the time column
df["hour"]=df["time"].str.replace(":.+","",regex=True).astype("int")
# Get the day of the week as a number from the 'date' column
df["dayCount"]=df["date"].dt.dayofweek
df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,dist,fk,pk,pkatt,season,team,venue_code,opponent_code,hour,dayCount
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0.0,1.0,Tottenham,...,16.9,1.0,0.0,0.0,2022,Manchester City,0,18,16,6
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5.0,0.0,Norwich City,...,17.3,1.0,0.0,0.0,2022,Manchester City,1,15,15,5
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5.0,0.0,Arsenal,...,14.3,0.0,0.0,0.0,2022,Manchester City,1,0,12,5
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1.0,0.0,Leicester City,...,14.0,0.0,0.0,0.0,2022,Manchester City,0,10,15,5
6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0.0,0.0,Southampton,...,15.7,1.0,0.0,0.0,2022,Manchester City,1,17,15,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2021-05-02,19:15,Premier League,Matchweek 34,Sun,Away,L,0.0,4.0,Tottenham,...,17.4,0.0,0.0,0.0,2021,Sheffield United,0,18,19,6
39,2021-05-08,15:00,Premier League,Matchweek 35,Sat,Home,L,0.0,2.0,Crystal Palace,...,11.4,1.0,0.0,0.0,2021,Sheffield United,1,6,15,5
40,2021-05-16,19:00,Premier League,Matchweek 36,Sun,Away,W,1.0,0.0,Everton,...,17.0,0.0,0.0,0.0,2021,Sheffield United,0,7,19,6
41,2021-05-19,18:00,Premier League,Matchweek 37,Wed,Away,L,0.0,1.0,Newcastle Utd,...,16.0,1.0,0.0,0.0,2021,Sheffield United,0,14,18,2


In [137]:
#convert the result column to a numeric value
df["target"]=(df["result"]=="W").astype("int")
df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,fk,pk,pkatt,season,team,venue_code,opponent_code,hour,dayCount,target
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0.0,1.0,Tottenham,...,1.0,0.0,0.0,2022,Manchester City,0,18,16,6,0
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5.0,0.0,Norwich City,...,1.0,0.0,0.0,2022,Manchester City,1,15,15,5,1
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5.0,0.0,Arsenal,...,0.0,0.0,0.0,2022,Manchester City,1,0,12,5,1
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1.0,0.0,Leicester City,...,0.0,0.0,0.0,2022,Manchester City,0,10,15,5,1
6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0.0,0.0,Southampton,...,1.0,0.0,0.0,2022,Manchester City,1,17,15,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2021-05-02,19:15,Premier League,Matchweek 34,Sun,Away,L,0.0,4.0,Tottenham,...,0.0,0.0,0.0,2021,Sheffield United,0,18,19,6,0
39,2021-05-08,15:00,Premier League,Matchweek 35,Sat,Home,L,0.0,2.0,Crystal Palace,...,1.0,0.0,0.0,2021,Sheffield United,1,6,15,5,0
40,2021-05-16,19:00,Premier League,Matchweek 36,Sun,Away,W,1.0,0.0,Everton,...,0.0,0.0,0.0,2021,Sheffield United,0,7,19,6,1
41,2021-05-19,18:00,Premier League,Matchweek 37,Wed,Away,L,0.0,1.0,Newcastle Utd,...,1.0,0.0,0.0,2021,Sheffield United,0,14,18,2,0


In [138]:
#500 individual decision trees we want to train
model=RandomForestClassifier(n_estimators=500,

    max_depth=5,min_samples_split=30,min_samples_leaf=15,

    random_state=42)


In [139]:
#making sure the training data set is older
train = df[df["date"] < '2022-01-01'].copy()
test = df[df["date"] > '2022-01-01'].copy()

In [140]:
pred=["venue_code","opponent_code","hour","dayCount"]

In [141]:
#training the mode with pred to be able to get the target
model.fit(train[pred],train["target"])

RandomForestClassifier(max_depth=5, min_samples_leaf=15, min_samples_split=30,
                       n_estimators=500, random_state=42)

In [142]:
predict=model.predict(test[pred])

In [143]:
accuracy_score(test["target"],predict)

0.6268115942028986

In [144]:
#combining actual and predited values
comp=pd.DataFrame(dict(actual=test["target"],predicted=predict))
pd.crosstab(index=comp["actual"],columns=comp["predicted"])

predicted,0,1
actual,,
0,165,7
1,96,8


In [145]:
precision_score(test["target"],predict)

0.5333333333333333

In [146]:
#to create a dataframe for each team we have
grouped=df.groupby("team")
group=grouped.get_group("Manchester United")

In [147]:
group

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,fk,pk,pkatt,season,team,venue_code,opponent_code,hour,dayCount,target
0,2021-08-14,12:30,Premier League,Matchweek 1,Sat,Home,W,5.0,1.0,Leeds United,...,0.0,0.0,0.0,2022,Manchester United,1,9,12,5,1
1,2021-08-22,14:00,Premier League,Matchweek 2,Sun,Away,D,1.0,1.0,Southampton,...,1.0,0.0,0.0,2022,Manchester United,0,17,14,6,0
2,2021-08-29,16:30,Premier League,Matchweek 3,Sun,Away,W,1.0,0.0,Wolves,...,1.0,0.0,0.0,2022,Manchester United,0,22,16,6,1
3,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Home,W,4.0,1.0,Newcastle Utd,...,0.0,0.0,0.0,2022,Manchester United,1,14,15,5,1
5,2021-09-19,14:00,Premier League,Matchweek 5,Sun,Away,W,2.0,1.0,West Ham,...,0.0,0.0,0.0,2022,Manchester United,0,21,14,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,2021-05-09,14:05,Premier League,Matchweek 35,Sun,Away,W,3.0,1.0,Aston Villa,...,0.0,1.0,1.0,2021,Manchester United,0,1,14,6,1
56,2021-05-11,18:00,Premier League,Matchweek 36,Tue,Home,L,1.0,2.0,Leicester City,...,0.0,0.0,0.0,2021,Manchester United,1,10,18,1,0
57,2021-05-13,20:15,Premier League,Matchweek 34,Thu,Home,L,2.0,4.0,Liverpool,...,1.0,0.0,0.0,2021,Manchester United,1,11,20,3,0
58,2021-05-18,18:00,Premier League,Matchweek 37,Tue,Home,D,1.0,1.0,Fulham,...,2.0,0.0,0.0,2021,Manchester United,1,8,18,1,0


In [148]:
def rollingAverage(group,cols,newCols):
    group=group.sort_values("date")
    rollingStats=group[cols].rolling(3,closed="left").mean()
    group[newCols]=rollingStats
    group=group.dropna(subset=new_cols)
    return group

In [149]:
cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt"]
new_cols = [f"{c}_rolling" for c in cols]
rollingAverage(group, cols, new_cols)

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,dayCount,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
5,2020-10-17,20:00,Premier League,Matchweek 5,Sat,Away,W,4.0,1.0,Newcastle Utd,...,5,1,1.666667,3.666667,9.000000,2.333333,19.800000,0.333333,0.666667,0.666667
7,2020-10-24,17:30,Premier League,Matchweek 6,Sat,Home,D,0.0,0.0,Chelsea,...,5,0,2.666667,3.000000,12.333333,4.666667,20.000000,0.000000,0.666667,1.000000
9,2020-11-01,16:30,Premier League,Matchweek 7,Sun,Home,L,0.0,1.0,Arsenal,...,6,0,1.666667,2.333333,15.000000,5.333333,19.700000,0.000000,0.333333,0.666667
11,2020-11-07,12:30,Premier League,Matchweek 8,Sat,Away,W,3.0,1.0,Everton,...,5,1,1.333333,0.666667,16.666667,5.666667,18.300000,0.000000,0.000000,0.333333
12,2020-11-21,20:00,Premier League,Matchweek 9,Sat,Home,W,1.0,0.0,West Brom,...,5,1,1.000000,0.666667,12.000000,3.666667,18.733333,0.666667,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40,2022-04-02,17:30,Premier League,Matchweek 31,Sat,Home,D,1.0,1.0,Leicester City,...,5,0,1.333333,2.000000,12.666667,3.666667,15.333333,0.333333,0.000000,0.000000
41,2022-04-09,12:30,Premier League,Matchweek 32,Sat,Away,L,0.0,1.0,Everton,...,5,0,1.666667,2.333333,9.000000,4.333333,14.333333,0.000000,0.000000,0.000000
42,2022-04-16,15:00,Premier League,Matchweek 33,Sat,Home,W,3.0,2.0,Norwich City,...,5,1,1.333333,1.333333,11.333333,5.000000,15.333333,0.000000,0.000000,0.000000
43,2022-04-19,20:00,Premier League,Matchweek 30,Tue,Away,L,0.0,4.0,Liverpool,...,1,0,1.333333,1.333333,14.333333,6.000000,15.666667,0.333333,0.000000,0.000000


In [150]:
#to compute the rolling averages function to each team
matchesroll = df.groupby("team").apply(lambda x: rollingAverage(x, cols, new_cols))


/tmp/ipython-input-531100016.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matchesroll = df.groupby("team").apply(lambda x: rollingAverage(x, cols, new_cols))


In [151]:
#dropping the team index to make it easier accessing each row
matchesroll = matchesroll.droplevel('team')


In [152]:
#assigning values from 0 to r (number of rows we have)
matchesroll.index = range(matchesroll.shape[0])


In [153]:
#retraining our model  with predictions func
def predictions(data, predictors):
    train = data[data["date"] < '2022-01-01']
    test = data[data["date"] > '2022-01-01']
    model.fit(train[predictors], train["target"])
    preds = model.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index=test.index)
    error = precision_score(test["target"], preds)
    return combined, error

In [154]:
# Ensure the predictors list is correctly formed before calling the function
predict = ["venue_code","opponent_code","hour","dayCount"]
predictors_list = predict + new_cols
combined, error = predictions(matchesroll, predictors_list)

In [155]:
precision_score(combined["actual"], combined["predicted"])

0.5384615384615384

In [156]:
combined

,actual,predicted
55,0,0
56,1,0
57,1,0
58,1,1
59,1,1
...,...,...
1312,1,0
1313,0,0
1314,1,0
1315,0,0


In [157]:
combined=combined.merge(matchesroll[["date","team","opponent","result"]],left_index=True,right_index=True)

In [158]:
combined

,actual,predicted,date,team,opponent,result
55,0,0,2022-01-23,Arsenal,Burnley,D
56,1,0,2022-02-10,Arsenal,Wolves,W
57,1,0,2022-02-19,Arsenal,Brentford,W
58,1,1,2022-02-24,Arsenal,Wolves,W
59,1,1,2022-03-06,Arsenal,Watford,W
...,...,...,...,...,...,...
1312,1,0,2022-03-13,Wolverhampton Wanderers,Everton,W
1313,0,0,2022-03-18,Wolverhampton Wanderers,Leeds United,L
1314,1,0,2022-04-02,Wolverhampton Wanderers,Aston Villa,W
1315,0,0,2022-04-08,Wolverhampton Wanderers,Newcastle Utd,L


In [159]:
#to replace any missing name with the original name
class MissingDict(dict):
    __missing__ = lambda self, key: key

map_values = {"Brighton and Hove Albion": "Brighton", "Manchester United": "Manchester Utd", "Newcastle United": "Newcastle Utd", "Tottenham Hotspur": "Tottenham", "West Ham United": "West Ham", "Wolverhampton Wanderers": "Wolves"}
mapping = MissingDict(**map_values)

In [160]:
combined["new_team"] = combined["team"].map(mapping)


In [161]:
#merging the df with itself
merged = combined.merge(combined, left_on=["date", "new_team"], right_on=["date", "opponent"])
merged

,actual_x,predicted_x,date,team_x,opponent_x,result_x,new_team_x,actual_y,predicted_y,team_y,opponent_y,result_y,new_team_y
0,0,0,2022-01-23,Arsenal,Burnley,D,Arsenal,0,0,Burnley,Arsenal,D,Burnley
1,1,0,2022-02-10,Arsenal,Wolves,W,Arsenal,0,0,Wolverhampton Wanderers,Arsenal,L,Wolves
2,1,0,2022-02-19,Arsenal,Brentford,W,Arsenal,0,0,Brentford,Arsenal,L,Brentford
3,1,1,2022-02-24,Arsenal,Wolves,W,Arsenal,0,0,Wolverhampton Wanderers,Arsenal,L,Wolves
4,1,1,2022-03-06,Arsenal,Watford,W,Arsenal,0,0,Watford,Arsenal,L,Watford
...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,1,0,2022-03-13,Wolverhampton Wanderers,Everton,W,Wolves,0,0,Everton,Wolves,L,Everton
258,0,0,2022-03-18,Wolverhampton Wanderers,Leeds United,L,Wolves,1,0,Leeds United,Wolves,W,Leeds United
259,1,0,2022-04-02,Wolverhampton Wanderers,Aston Villa,W,Wolves,0,0,Aston Villa,Wolves,L,Aston Villa
260,0,0,2022-04-08,Wolverhampton Wanderers,Newcastle Utd,L,Wolves,1,0,Newcastle United,Wolves,W,Newcastle Utd


In [162]:
merged[(merged["predicted_x"] == 1) & (merged["predicted_y"] ==0)]["actual_x"].value_counts()


,count
actual_x,
1,13
0,10


In [163]:
accuracy_score(merged["actual_x"], merged["predicted_x"])*100


61.45038167938931

In [164]:
precision_score(merged["actual_x"], merged["predicted_x"])*100

56.00000000000001